# 10.6 LightGBM算法案例实战2 - 广告收益回归预测模型

**10.6.1 案例背景**

投资商经常会通过多个不同渠道投放广告，以此来获得经济利益。在本案例中我们选取公司在电视、广播和报纸上的投入，来预测广告收益，这对公司策略的制定是有较重要的意义。

**10.6.2 模型搭建**

In [2]:
# 读取数据
import pandas as pd
df = pd.read_excel('广告收益数据.xlsx')
df.head()

,电视,广播,报纸,收益
0,230.1,37.8,69.2,331.5
1,44.5,39.3,45.1,156.0
2,17.2,45.9,69.3,139.5
3,151.5,41.3,58.5,277.5
4,180.8,10.8,58.4,193.5


In [3]:
# 1.提取特征变量和目标变量
X = df.drop(columns='收益') 
y = df['收益']  

# 2.划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# 3. 模型训练和搭建
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)

LGBMRegressor()

**10.6.3 模型预测及评估**

In [4]:
# 预测测试数据
y_pred = model.predict(X_test)
y_pred[0:5]

array([192.6139063 , 295.11999665, 179.92649365, 293.45888909,
       166.86159398])

In [4]:
# 预测值和实际值对比
a = pd.DataFrame()  # 创建一个空DataFrame 
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,192.613906,190.5
1,295.119997,292.5
2,179.926494,171.0
3,293.458889,324.0
4,166.861594,144.0


In [5]:
# 手动输入数据进行预测
X = [[71, 11, 2]]
model.predict(X)

array([140.00420258])

In [6]:
# 查看R-square
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(X_test))
r2

0.9570203214455993

In [7]:
# 查看评分
model.score(X_test, y_test)

0.9570203214455993

In [7]:
# 特征重要性
model.feature_importances_

array([ 950, 1049,  963])

**10.6.4 模型参数调优**

In [8]:
# 参数调优
from sklearn.model_selection import GridSearchCV  # 网格搜索合适的超参数
parameters = {'num_leaves': [15, 31, 62], 'n_estimators': [20, 30, 50, 70], 'learning_rate': [0.1, 0.2, 0.3, 0.4]}  # 指定分类器中参数的范围
model = LGBMRegressor()  # 构建模型
grid_search = GridSearchCV(model, parameters,scoring='r2',cv=5) # cv=5表示交叉验证5次，scoring='r2'表示以R-squared作为模型评价准则

In [9]:
# 输出参数最优值
grid_search.fit(X_train, y_train)  # 传入数据
grid_search.best_params_  # 输出参数的最优值

{'learning_rate': 0.3, 'n_estimators': 50, 'num_leaves': 31}

In [11]:
# 重新搭建LightGBM回归模型
model = LGBMRegressor(num_leaves=31, n_estimators=50,learning_rate=0.3)
model.fit(X_train, y_train)

# 查看得分
model.score(X_test, y_test)

0.9558624845475153